In [1]:
from VoxResNetPytorch import *

In [2]:
vox_net = VoxResNet()

In [3]:
vox_net

VoxResNet(
  (model): Sequential(
    (conv3d_1): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    (batch_norm_1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True)
    (activation_1): ReLU(inplace)
    (conv3d_2): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (batch_norm_2): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True)
    (activation_2): ReLU(inplace)
    (conv3d_3): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    (block_1): BasicBlock(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (block_2): BasicBlock(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3),